In [1]:
import pandas as pd

main_file = "data.parquet"
df = pd.read_parquet(main_file, engine='auto')

# divide collumn tpep_dropoff_datatime into date and time respectively
df[['tpep_dropoff_date', 'tpep_dropoff_time']] = df['tpep_dropoff_datetime'].astype(str).str.split(' ', 1, expand = True)

# delete the excessive tpep_dropoff_time collumn
del df['tpep_dropoff_time']

# divide the file based on tpep_dropoff_date:
df.to_parquet('./partitioned', partition_cols=['tpep_dropoff_date'])

In [2]:
# view how the main file was read
#df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,tpep_dropoff_date
0,1,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,N,238,239,1,6.00,3.00,0.5,1.47,0.00,0.3,11.27,2.5,None,2020-01-01
1,1,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.20,1.0,N,239,238,1,7.00,3.00,0.5,1.50,0.00,0.3,12.30,2.5,None,2020-01-01
2,1,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.60,1.0,N,238,238,1,6.00,3.00,0.5,1.00,0.00,0.3,10.80,2.5,None,2020-01-01
3,1,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.80,1.0,N,238,151,1,5.50,0.50,0.5,1.36,0.00,0.3,8.16,0.0,None,2020-01-01
4,2,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.00,1.0,N,193,193,2,3.50,0.50,0.5,0.00,0.00,0.3,4.80,0.0,None,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6405003,2,2020-01-31 23:08:00,2020-01-31 23:34:00,NaN,12.07,NaN,None,191,35,0,52.18,2.75,0.5,0.00,0.00,0.3,55.73,NaN,None,2020-01-31
6405004,2,2020-01-31 23:00:00,2020-01-31 23:44:00,NaN,23.48,NaN,None,186,117,0,54.01,2.75,0.5,0.00,6.12,0.3,63.68,NaN,None,2020-01-31
6405005,2,2020-01-31 23:13:00,2020-01-31 23:38:00,NaN,12.09,NaN,None,164,131,0,38.02,2.75,0.5,0.00,6.12,0.3,47.69,NaN,None,2020-01-31
6405006,2,2020-01-31 23:02:54,2020-01-31 23:27:10,NaN,11.65,NaN,None,10,16,0,28.74,2.75,0.0,0.00,0.00,0.3,31.79,NaN,None,2020-01-31


In [3]:
import glob

# Select which files will be reuploaded based on their dropoff date:
part_files = "./partitioned/*2020-01*/*"

# Re-read files, but only with expected tpep_dropoff_datetime:
newdf = pd.read_parquet(glob.glob(part_files), engine='auto')

In [ ]:
# Uncomment to view how the files have been read after eliminating anomalies
#newdf

In [4]:
#compare the number of rows that we started with, to the number of rows that were re-read:
print("Original rows: ", len(df.index))
print("Re-read rows:  ", len(newdf.index))

Original rows:  6405008
Re-read rows:   6402165


In [5]:
import sqlite3
import sqlalchemy

In [6]:
engine = sqlalchemy.create_engine('sqlite:///DataBase.db')

In [7]:
newdf.to_sql('table', engine, if_exists = 'append', index = False, chunksize = 1000, method = None)

6402165